In [ ]:
from sklearn.preprocessing import MinMaxScaler
import itertools
import pandas as pd
import torch

# get our predefined functions (for data processing, etc.)
import QCircNet.utils as ut

# get our predefined quantum circuits and neural networks
import QCircNet.circuits.CustomCircuits as cn
import QCircNet.QuantumNN as qnn


EXPERIMENT_ID = 69

DATA_PATH = "data/"


N_QUBITS = 4
FEAT_PER_QUBIT = 4

SUBSET = 50

EPOCHS = 2
WEIGHT_PATH = f"results/weights/"
MODEL_PATH = f"results/models/"
RESULTS_PATH = f"results/model_10_results_expID_{EXPERIMENT_ID}.csv"
RESULTS_DF = pd.DataFrame()

# for printing colored text output during training
RED_TEXT = "\033[91m"
GREEN_TEXT = "\033[92m"
RESET_COLOR = "\033[0m" 


In [2]:
# load the data
X_train, y_train, X_test, y_test  = ut.load_MNIST_data(DATA_PATH, subset=SUBSET)

# check shapes
print("train:\t", X_train.shape, y_train.shape)
print("test:\t", X_test.shape, y_test.shape)

train:	 torch.Size([50, 16]) torch.Size([50, 10])
test:	 torch.Size([50, 16]) torch.Size([50, 10])


In [3]:
cn.get_custom_circuits()

[QCircNet.circuits.CustomCircuits.Anusha2024Circuit,
 QCircNet.circuits.CustomCircuits.DoubleEntanglementVLCircuit,
 QCircNet.circuits.CustomCircuits.Ranga2024Circuit,
 QCircNet.circuits.CustomCircuits.Senokosov2024Circuit]

In [4]:
circuits = [
    # cn.Ranga2024Circuit,
    cn.Senokosov2024Circuit,
    # cn.Anusha2024Circuit,
    # cn.DoubleEntanglementVLCircuit,
]

runs = list(range(1, 2)) # runs are also used as seeds (reproducibility)
learning_rates = [0.01, 0.001]

In [ ]:
### RUN EXPERIMENT

id = 0
for seed, circuit, lr in itertools.product(runs, circuits, learning_rates):
    print(f"""{RED_TEXT}RUN {id+1}:{RESET_COLOR}\n {circuit.__name__} \n lr: {lr}""")

    # init the model
    model = qnn.QuantumClassifierMNIST(n_qubits=N_QUBITS, features_per_qubit=FEAT_PER_QUBIT, circuit=circuit, seed=seed)

    path_identifier = f"{circuit.__name__}_expID_{EXPERIMENT_ID}_dfID_{id}"

    # train the model (predifined optimizer, etc.)
    trained_model, loss_history, avg_epoch_time = ut.train_model(
        model, X_train, y_train,
        epochs=EPOCHS, batch_size=32, lr=lr, binary=True,
        weight_path=WEIGHT_PATH
    )
    
    # save trained model
    model_path = MODEL_PATH+path_identifier+".pth"
    torch.save(trained_model, model_path)

    # evaluate
    cross_entropy, accuracy, precision, recall, f1 = trained_model.evaluate(X_test, y_test)

    results_dict = {
        "id": id,
        "f1_score": f1, "precision": precision, "recall": recall, "accuracy": accuracy, "cross_entropy": cross_entropy,
        "seed": seed,
        "circuit": circuit.__name__,
        "learning_rate": lr,
        "loss_history": loss_history,
        "avg_epoch_time_seconds": avg_epoch_time,
        "network_str": str(model),
        "model_path": model_path,
        "weight_path": WEIGHT_PATH,
        "epochs": EPOCHS,
        "n_qubits": N_QUBITS,
        "features_per_qubit": FEAT_PER_QUBIT,
        # "scaler" : str(SCALER).replace("()", "")
    }
    print(f"\n{GREEN_TEXT}RESULTS:\n F1-Score:{f1}, Precision:{precision}, Recall:{recall}, Accuracy:{accuracy}, Loss:{cross_entropy} {RESET_COLOR} \n\n")
    
    # append the results_dict to the RESULTS_DF
    RESULTS_DF = pd.concat([RESULTS_DF, pd.DataFrame([results_dict])], ignore_index=True)
    RESULTS_DF.to_csv(path_or_buf=RESULTS_PATH) # save for each run (in case of a crash)

    id += 1

RUN 1:
 Senokosov2024Circuit 
 lr: 0.01


Training progress: 100%|██████████| 1/1 [00:00<00:00,  1.72batch/s]


RuntimeError: Parent directory results/models_10/Senokosov2024Circuit_expID_69_dfID_0.pth does not exist.

In [ ]:
RESULTS_DF

,id,f1_score,precision,recall,accuracy,cross_entropy,seed,circuit,learning_rate,loss_history,avg_epoch_time_seconds,network_str,model_path,weights,epochs,n_qubits,features_per_qubit
0,0,0.113208,1.0,0.06,0.06,2.203273,1,Senokosov2024Circuit,0.010,"[0.2503029704093933, 0.2477768361568451]",0.595407,QuantumClassifierMNIST(\n (quantum_circuit_nn...,results/models_10/model_expID_69_dfID_0.pth,[],2,4,4
1,1,0.000000,0.0,0.00,0.00,2.210763,1,Senokosov2024Circuit,0.001,"[0.2503029704093933, 0.25011005997657776]",0.545868,QuantumClassifierMNIST(\n (quantum_circuit_nn...,results/models_10/model_expID_69_dfID_1.pth,[],2,4,4


In [ ]:
# circuit = qc.QuantumCircuit

# # initialize the model
# model = qnn.QuantumNeuralNetwork(n_qubits=N_QUBITS, features_per_qubit=FEAT_PER_QUBIT, circuit=circuit, seed=1)

# # train the model (predifined optimizer, etc.)
# trained_model, loss_history, val_loss_history = ut.train_model(
#     model, X_train, y_train, epochs=EPOCHS, batch_size=64, lr=0.01
# ) #X_val, y_val,


# # Evaluate the model
# bce, accuracy, precision, recall, f1 = trained_model.evaluate(X_test, y_test)
# print(f"Binary Cross-Entropy on test data: {bce:.4f}")
# print(f"Accuracy: {accuracy:.4f}")
# print(f"Precision: {precision:.4f}")
# print(f"Recall: {recall:.4f}")
# print(f"F1 Score: {f1:.4f}")

In [ ]:
# # To load later
# loaded_model = torch.load("simple_model.pth")